In [13]:
import os
import requests

# Load the API key from an environment variable
API_KEY = os.getenv('CORE_API_KEY')
BASE_URL = 'https://api.core.ac.uk/v3'

def fetch_data(endpoint, params=None):
    """
    Fetches data from the specified CORE API endpoint.
    
    Args:
        endpoint (str): The endpoint of the API to call.
        params (dict): Optional dictionary of parameters to include in the request.
        
    Returns:
        dict: The JSON response from the API as a dictionary.
    """
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        # Raise an error if the request was not successful
        response.raise_for_status()

def get_single_research(query):
    """
    Retrieves a single research article with selected metadata fields.
    
    Args:
        query (str): The search query (e.g., "machine learning").
        
    Returns:
        dict: Selected metadata fields of the first research article that matches the search query.
    """
    search_params = {
        'q': query,
        'page': 1,
        'pageSize': 1
    }
    
    # Search for articles
    search_results = fetch_data('search/works', params=search_params)
    
    # Extract the first result's ID
    if 'results' in search_results and len(search_results['results']) > 0:
        article_id = search_results['results'][0]['id']
        
        # Fetch full metadata and text for the article
        article_data = fetch_data(f'works/{article_id}')
        
        # Extract specific fields
        title = article_data.get('title', 'No title available')
        authors = [author.get('name', 'Unknown') for author in article_data.get('authors', [])]
        year = article_data.get('year', 'No year available')
        abstract = article_data.get('abstract', 'No abstract available')
        full_text = article_data.get('fullText', 'No full text available')
        full_text_link = article_data.get('downloadUrl', 'No link available')
        
        # Format and return selected metadata fields
        return {
            "Title": title,
            "Authors": ", ".join(authors),
            "Year": year,
            "Abstract": abstract,
            "Full Text": full_text[:1000] + '...' if len(full_text) > 1000 else full_text,  # Truncate for display
            "Link": full_text_link
        }
    else:
        print("No results found for the query.")
        return None

# Example usage of the function
if __name__ == "__main__":
    query = "machine learning"  # Change the query as needed
    article = get_single_research(query)
    
    if article:
        print("Selected Metadata:")
        for key, value in article.items():
            print(f"{key}: {value}")


Selected Metadata:
Title: A Hybrid Approach to Audio-to-Score Alignment
Authors: Agrawal, R, Dixon, S, Machine Learning for Music Discovery Workshop at International Conference on Machine Learning (ICML)
Year: No year available
Abstract: None
Full Text: A Hybrid Approach to Audio-to-Score AlignmentRuchit Agrawal 1 Simon Dixon 1AbstractAudio-to-score alignment aims at generating anaccurate mapping between a performance audioand the score of a given piece. Standard align-ment methods are based on Dynamic Time Warp-ing (DTW) and employ handcrafted features. Weexplore the usage of neural networks as a prepro-cessing step for DTW-based automatic alignmentmethods. Experiments on music data from dif-ferent acoustic conditions demonstrate that thismethod generates robust alignments whilst beingadaptable at the same time.1. Introduction and MotivationAudio-to-score alignment is the task of finding the optimalmapping between a performance and the score for a givenpiece of music. Dynamic Time War